[View in Colaboratory](https://colab.research.google.com/github/moltal/gravitiy_wave/blob/master/%E3%83%81%E3%83%A5%E3%83%BC%E3%83%88%E3%83%AA%E3%82%A2%E3%83%AB.ipynb)

#LIGOのデータを用いた重力波の解析


---
##はじめに
2016年2月11日重力波望遠鏡LIGOが重力波の存在を証明する事象を検出することができたと発表した。
##重力波とは
物質はその質量によって時空を歪める。これはブラックホールなどの質量が大きな物質ほど顕著である。物質が動いていると、変動する歪みが進んでいく。この歪みが重力波である。  アインシュタイン方程式によって
$$$$
##LIGOとは

##GW150914とは
　GW150914とは'(20)15(年)09(月)14(日)'にLIGOによって初めて検出された重力波'Gravitational Wave (GW)'につけられた名称である。このイベントはLIGOが改良後の観測運転を始めた途端に観測され、2月11日に発表された。13億年前に

##GW150914の解析
今回はthe LIGO Open Science Center (LOSC)が実施しているチュートリアルに従って解析を行った。
https://losc.ligo.org/s/events/GW170104/LOSC_Event_tutorial_GW170104.html

###モジュールのインポート

In [0]:
#GW150914について解析を行う
eventname = 'GW150914' 

make_plots = 1
plottype = "png"

In [5]:
#Google Colablatoryにデータを移行
from google.colab import files
uploaded = files.upload()

Saving BBH_events_v3.json to BBH_events_v3.json
Saving GW150914_4_template.hdf5 to GW150914_4_template.hdf5
Saving H-H1_LOSC_4_V1-1167559920-32.hdf5 to H-H1_LOSC_4_V1-1167559920-32.hdf5
Saving L-L1_LOSC_4_V1-1167559920-32.hdf5 to L-L1_LOSC_4_V1-1167559920-32.hdf5
Saving LVT151012_4_template.hdf5 to LVT151012_4_template.hdf5
Saving readligo.py to readligo.py


In [0]:
# 標準の数的解析のモジュールをインポート
import numpy as np
from scipy import signal
from scipy.interpolate import interp1d
from scipy.signal import butter, filtfilt, iirdesign, zpk2tf, freqz
import h5py
import json


%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab


# LIGO-specific readligo.py 

import readligo as rl


### json fileからイベントプロパティを読み込む

In [0]:
# local json fileからイベントプロパティを読み込む

fnjson = "BBH_events_v3.json"
try:
    events = json.load(open(fnjson,"r"))
except IOError:
    print("Cannot find resource file "+fnjson)
    print("You can download it from https://losc.ligo.org/s/events/"+fnjson)
    print("Quitting.")
    quit()

# イベントネームを設定してない場合
try: 
    events[eventname]
except:
    print('You must select an eventname that is in '+fnjson+'! Quitting.')
    quit()

In [36]:
# 選んだイベントの値を変数に代入する
event = events[eventname]
fn_H1 = event['fn_H1']              # H1(ハンフォード観測所)でのデータ
fn_L1 = event['fn_L1']              # L１(リビングストン観測所)でのデータ
fn_template = event['fn_template']  # 予想されるテンプレートのデータ
fs = event['fs']                    # サンプルレート
tevent = event['tevent']            # イベントのおおよそのGPS時刻
fband = event['fband']              # バンドパスシグナルの周波数帯
print("Reading in parameters for event " + event["name"])
print(event)

Reading in parameters for event GW150914
{'name': 'GW150914', 'fn_H1': 'H-H1_LOSC_4_V2-1126259446-32.hdf5', 'fn_L1': 'L-L1_LOSC_4_V2-1126259446-32.hdf5', 'fn_template': 'GW150914_4_template.hdf5', 'fs': 4096, 'tevent': 1126259462.44, 'utcevent': '2015-09-14T09:50:45.44', 'm1': 41.743, 'm2': 29.237, 'a1': 0.355, 'a2': -0.769, 'approx': 'lalsim.SEOBNRv2', 'fband': [43.0, 300.0], 'f_min': 10.0}


###データを読み込む


In [37]:
#----------------------------------------------------------------
#　LOSCデータを読み込む
# はじめにfn_H1,fn_L1の中身を定義する。
#----------------------------------------------------------------
try:
    # H1,L1のデータを読み込む
    strain_H1, time_H1, chan_dict_H1 = rl.loaddata(fn_H1, 'H1')
    strain_L1, time_L1, chan_dict_L1 = rl.loaddata(fn_L1, 'L1')
    print(chan_dict_H1)
except:
    print("Cannot find data files!")
    print("You can download them from https://losc.ligo.org/s/events/"+eventname)
    print("Quitting.")
    quit()

None


In [38]:
# H1とL1は同じ時間
time = time_H1
print(time_H1)
# サンプルのdtを定義
dt = time[1] - time[0]

# 各データのデータ数、最小値、平均値、最大値

print('time_H1: len, min, mean, max = ', \
    len(time_H1), time_H1.min(), time_H1.mean(), time_H1.max() )
print('strain_H1: len, min, mean, max = ', \
    len(strain_H1), strain_H1.min(),strain_H1.mean(),strain_H1.max())
print( 'strain_L1: len, min, mean, max = ', \
    len(strain_L1), strain_L1.min(),strain_L1.mean(),strain_L1.max())

bits = chan_dict_H1['DATA']
print("For H1, {0} out of {1} seconds contain usable DATA".format(bits.sum(), len(bits)))
bits = chan_dict_L1['DATA']
print("For L1, {0} out of {1} seconds contain usable DATA".format(bits.sum(), len(bits)))

None


TypeError: ignored

In [16]:
#  +- deltat秒の時間でプロットする
deltat = 5
indxt = np.where((time >= tevent-deltat) & (time < tevent+deltat))
print(tevent)

if make_plots:
    plt.figure()
    plt.plot(time[indxt]-tevent,strain_H1[indxt],'r',label='H1 strain')
    plt.plot(time[indxt]-tevent,strain_L1[indxt],'g',label='L1 strain')
    plt.xlabel('time (s) since '+str(tevent))
    plt.ylabel('strain')
    plt.legend(loc='lower right')
    plt.title('Advanced LIGO strain data near '+eventname)
    plt.savefig(eventname+'_strain.'+plottype)

TypeError: ignored

赤線がハンフォード観測所で観測された波形(H1 strain)  
緑線がリビングストン観測所で観測された波形(L1 strain)  

In [17]:
make_psds = 1
if make_psds:
    # number of sample for the fast fourier transform:
    NFFT = 4*fs
    Pxx_H1, freqs = mlab.psd(strain_H1, Fs = fs, NFFT = NFFT)
    Pxx_L1, freqs = mlab.psd(strain_L1, Fs = fs, NFFT = NFFT)

    # We will use interpolations of the ASDs computed above for whitening:
    psd_H1 = interp1d(freqs, Pxx_H1)
    psd_L1 = interp1d(freqs, Pxx_L1)

    # Here is an approximate, smoothed PSD for H1 during O1, with no lines. We'll use it later.    
    Pxx = (1.e-22*(18./(0.1+freqs))**2)**2+0.7e-23**2+((freqs/2000.)*4.e-23)**2
    psd_smooth = interp1d(freqs, Pxx)

if make_plots:
    # plot the ASDs, with the template overlaid:
    f_min = 20.
    f_max = 2000. 
    plt.figure(figsize=(10,8))
    plt.loglog(freqs, np.sqrt(Pxx_L1),'g',label='L1 strain')
    plt.loglog(freqs, np.sqrt(Pxx_H1),'r',label='H1 strain')
    plt.loglog(freqs, np.sqrt(Pxx),'k',label='H1 strain, O1 smooth model')
    plt.axis([f_min, f_max, 1e-24, 1e-19])
    plt.grid('on')
    plt.ylabel('ASD (strain/rtHz)')
    plt.xlabel('Freq (Hz)')
    plt.legend(loc='upper center')
    plt.title('Advanced LIGO strain data near '+eventname)
    plt.savefig(eventname+'_ASDs.'+plottype)

TypeError: ignored

##参考文献
BINARY BLACK HOLE SIGNALS IN LIGO OPEN DATA(Version 1.63, 2017 Sept 11), LOSC.
https://losc.ligo.org/s/events/GW170104/LOSC_Event_tutorial_GW170104.html

重力波で見える宇宙の始まり, ピエール・ビネトリュイ(2017), 国宝社.